In [ ]:
import requests
import json
from urllib.parse import quote

# WebサーバのAPIエンドポイント
url = "https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q="

# エンドポイントにPOSTリクエストを送信
def _run(url, query: str):
    query2 = url + quote(query)  #.replace("'", '"').encode('utf-8')
    print("くえり", query2)
    # query_dict = json.loads(query)
    response = requests.get(query2)  # .post(url) # +  "get_topic", query_dict)
    print(response)
    topic_content = response.text
    print(topic_content)
    return topic_content

result = _run(url, "大規模言語モデル")
data = json.loads(result)  

!conda list jq
import jq
だめらしい
Failed to build jq
ERROR: Could not build wheels for jq, which is required to install pyproject.toml-based projects
ので
https://github.com/langchain-ai/langchain/issues/4396

In [1]:
# https://github.com/langchain-ai/langchain/issues/4396
import json
from pathlib import Path
from typing import Callable, Dict, List, Optional, Union
import requests
import urllib.parse

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader

import pprint

class JSONLoader(BaseLoader):
    def __init__(
        self , 
        # file_path: Union[str, Path],
        file_path: Path,
        content_key: Optional[str] = None,
        metadata: Optional[str] = None,
        ):
        # self.file_path = Path(file_path).resolve()
        self.file_path = file_path
        self._content_key = content_key
        self.metadata_item = metadata

    def create_documents(self,processed_data):
    # def create_documents(processed_data):
        documents = []
        for item in processed_data:
            if '@id: ' in item:  # 'link.@id: '
                # print(item)
                self.metadata_item = ''.join(item)
            content = ''.join(item)
            # print(self.metadata_item)
            document = Document(page_content=content, metadata={'@id':self.metadata_item})
            documents.append(document)
        return documents
    
    def process_item(self, item, prefix=""):
        # print(item)
        if isinstance(item, dict):
            result = []
            for key, value in item.items():
                new_prefix = f"{prefix}.{key}" if prefix else key
                result.extend(self.process_item(value, new_prefix))
            return result
        elif isinstance(item, list):
            result = []
            for value in item:
                result.extend(self.process_item(value, prefix))
            return result
        else:
            return [f"{prefix}: {item}"]

    def process_json(self,data):
        if isinstance(data, list):
            processed_data = []
            # print('process_json_data', data)
            for item in data:
                processed_data.extend(self.process_item(item))
            return processed_data
        elif isinstance(data, dict):
            return self.process_item(data)
        else:
            return []

    def load(self) -> List[Document]:
        """Load and return documents from the JSON file."""

        docs=[]
        # with self as json_file:
        # with open(self.file_path, 'r') as json_file:
        try:
                # print('self.file_path', self.file_path)
                # data = json.load(self.file_path)  # (json_file)
                # print(json.loads(self))
                # print('data',data)
                data = self.file_path
                processed_json = self.process_json(data) # json.process_json(self.file_path)  #(data)
                # print(processed_json)
                docs = self.create_documents(processed_json)
                # print(docs)
        except json.JSONDecodeError:
                print("Error: Invalid JSON format in the file.")
        return docs

def filter_function(item):
    return item['items']  #[0] #["dc:type"] == "Article"# item['age'] >= 20 and item['city'] == 'Tokyo'

url = 'https://cir.nii.ac.jp/opensearch/all?sortorder=0&format=json&'
d = {'count': '10', 'q': '大規模言語モデル'}
d_qs = url + urllib.parse.urlencode(d)

response = requests.get(d_qs)  
#('https://cir.nii.ac.jp/opensearch/all?count=5&sortorder=0&format=json&q=%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB%20%E5%BF%9C%E7%94%A8')
dataaa = response.json()
filtered_data = filter_function(dataaa)

# pprint.pprint(filtered_data)  # dataaa)

loader = JSONLoader(filtered_data)  # (dataaa) #(response) # (dataaa)  # (file_path=file_path)
# print(loader)
data = loader.load()
pprint.pprint(data)

# [Document(page_content='@id: https://cir.nii.ac.jp/crid/1520016748559385472'),
#  Document(page_content='@type: item'), .....

[Document(page_content='@id: https://cir.nii.ac.jp/crid/1390298668092493184', metadata={'@id': '@id: https://cir.nii.ac.jp/crid/1390298668092493184'}),
 Document(page_content='@type: item', metadata={'@id': '@id: https://cir.nii.ac.jp/crid/1390298668092493184'}),
 Document(page_content='title: 生成AIがやってきた！東北大学における注意喚起発出の経緯と方針，そして…', metadata={'@id': '@id: https://cir.nii.ac.jp/crid/1390298668092493184'}),
 Document(page_content='link.@id: https://cir.nii.ac.jp/crid/1390298668092493184', metadata={'@id': 'link.@id: https://cir.nii.ac.jp/crid/1390298668092493184'}),
 Document(page_content='rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json'}),
 Document(page_content='dc:creator: 三石 大', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json'}),
 Document(page_content='dc:publisher: 教育システム情報学会', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.j

In [ ]:
print(len(documents))
documents[0]

In [7]:
# embeddingの実行
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import os

documents = data

def database(**args_dict):
    print(args_dict['db'], args_dict['embedding_function'])
    # create the open-source embedding function
    embedding_function = SentenceTransformerEmbeddings(
        model_name = args_dict['embedding_function'],
        encode_kwargs={"normalize_embeddings":False })
    path = os.path.join('./database2/', args_dict['db'])
    print(path)
    db = Chroma.from_documents(
        documents=documents,
        embedding = embedding_function,
        persist_directory = path
    )
    if db:
        db.persist()
        # db = None
    else:
        print("Chroma DB has not been initialized.")

list_db = [
    {'db':'db1', 'embedding_function':'sentence-transformers/all-MiniLM-L6-v2'},
    {'db':'db2', 'embedding_function':'oshizo/sbert-jsnli-luke-japanese-base-lite'},
    {'db':'db3', 'embedding_function':'intfloat/multilingual-e5-small'},
    {'db':'db4', 'embedding_function':'sonoisa/sentence-bert-base-ja-mean-tokens-v2'}
]

for dict_db in list_db:
    database(**dict_db)

db1 sentence-transformers/all-MiniLM-L6-v2
./database2/db1
db2 oshizo/sbert-jsnli-luke-japanese-base-lite
./database2/db2
db3 intfloat/multilingual-e5-small
./database2/db3
db4 sonoisa/sentence-bert-base-ja-mean-tokens-v2
./database2/db4


In [4]:
# 使わない（もともと最初に作ったやつ）
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

db = Chroma.from_documents(
    documents=data,    # documents,
    #  InvalidDimensionException: Dimensionality of (1536) does not match index dimensionality (384)
    # embeddingを変更したらいったんDBのフォルダごと削除
    # embedding=OpenAIEmbeddings(),
    # embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite" , encode_kwargs={"normalize_embeddings":True }),
    embedding = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-small', 
                                      encode_kwargs={"normalize_embeddings":True }),
    persist_directory='./database2/'
)
if db:
    db.persist()
    # db = None
else:
    print("Chroma DB has not been initialized.")

# c:\Users\onowa\miniforge3\envs\env_chroma\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
#  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# 一応、これで完成？
# dbs = ['db1','db2','db3','db4']
# データベースをディスクからロードして、クエリで検索する

list_db = [
    {'db':'db1', 'embedding_function':'sentence-transformers/all-MiniLM-L6-v2'},
    {'db':'db2', 'embedding_function':'oshizo/sbert-jsnli-luke-japanese-base-lite'},
    {'db':'db3', 'embedding_function':'intfloat/multilingual-e5-small'},
    {'db':'db4', 'embedding_function':'sonoisa/sentence-bert-base-ja-mean-tokens-v2'}
]

query ='教育情報'
top_k=10

def load_from_disk(**args_dict):
    print(args_dict['db'], args_dict['embedding_function'])
    path = os.path.join('./database2/', args_dict['db'])
    # print(path)
    embedding_function = SentenceTransformerEmbeddings(model_name=args_dict['embedding_function'])
    db = Chroma(persist_directory = path, embedding_function=embedding_function)
    docs = db.similarity_search_with_score(query, K=top_k)
    # docs = db.similarity_search(query, k=top_k)
    # print(docs[0].page_content)
    print(len(docs))
    # similarity_search_with_score の場合：
    # print(f"content: {docs[0][0].page_content}", f"score: {docs[0][1]}")  
    # print(f"content: {docs[3][0].page_content}", f"score: {docs[1][1]}")  
    for mykey in docs:
        print(mykey)
    print('------\n')

for each_db in list_db:
    load_from_disk(**each_db)

db1 sentence-transformers/all-MiniLM-L6-v2
4
(Document(page_content='title: キャラクタ対話システムのための文脈を用いた応答評価', metadata={'@id': '@id: https://cir.nii.ac.jp/crid/1390579830889855744'}), 1.272719383239746)
(Document(page_content='title: Remdis: リアルタイムマルチモーダル対話システム構築ツールキット', metadata={'@id': '@id: https://cir.nii.ac.jp/crid/1390016880936443648'}), 1.2835525274276733)
(Document(page_content='dc:subject: 注意喚起', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json'}), 1.3283870220184326)
(Document(page_content='dc:subject: 教育応用', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json'}), 1.3283870220184326)
------

db2 oshizo/sbert-jsnli-luke-japanese-base-lite
4
(Document(page_content='prism:publicationName: 教育システム情報学会誌', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.jp/crid/1390298668092493184.json'}), 54.82743453979492)
(Document(page_content='dc:subject: 教育現場対応', metadata={'@id': 'rdfs:seeAlso.@id: https://cir.nii.ac.j